In [ ]:
---
layout: post
title: CSSE2 Final ( Work )
description: Hope's work in CSSE2 in the RPG game and the Platform games
categories: [Personal]
permalink: /personal/CSSE2-Final/
type: collab
comments: true
---


## Collision in Overworld and Water 

Since these two are very similar, this part will be going over them both. 

I was responsible for handling the collision on both levels in order for the player and enemies to collide. With the collision, that would lead to the player dying and the level having to be restarted. In our own game level, there is an enemy Creeper walking around the level. Since we wanted something to happen when the player touches (collides) with the Creeper, we created 'enemy.js', which has it's own collision checks. 

```js 
update() {
    this.draw(); // First, the enemy draws itself
    if (!this.playerDestroyed && this.collisionChecks()) {
        this.handleCollisionEvent();
    }
    this.stayWithinCanvas(); // Prevents the enemy from moving outside canvas
}
```
In each frame, the Enemy updates its visuals ``(this.draw())``.

Then, if the enemy hasn’t already “killed” the player ``(!this.playerDestroyed)``, it checks for a collision using ``collisionChecks()``.

If there’s a collision, it calls ``handleCollisionEvent()``.

```js 
collisionChecks() {
    for (const gameObj of this.gameEnv.gameObjects) {
        if (gameObj instanceof Player) {
            this.isCollision(gameObj);
            if (this.collisionData.hit) {
                return true;
            }
        }
    }
    return false;
}
```

The enemy loops through all game objects in the environment ``(this.gameEnv.gameObjects)``.

- If it finds a Player object, it calls ``this.isCollision(gameObj)``.
- ``this.isCollision()`` determines whether the bounding boxes of the enemy and the player overlap.
- If ``this.collisionData.hit`` is set to true, a collision is confirmed.

```js 
handleCollisionEvent() {
    console.log("Player collided with the Enemy. Player is dead.");
    this.playerDestroyed = true; // Mark the player as "dead"
    alert("KABOOM!!");
    this.gameEnv.gameControl.currentLevel.restart = true; 
}
```

The enemy logs to the console and shows an alert to signal the collision.

- It sets ``this.playerDestroyed`` to true (so it doesn’t kill the player repeatedly!).
- It sets the current level’s ``restart`` flag to true, signaling the game to restart the level.

#### In summary

The collision detection is done by calling ``this.isCollision()`` on the Player object, and checking if ``this.collisionData.hit`` is true.
The reaction to the collision (like killing the player, triggering level restart, and explosion) is done in ``handleCollisionEvent()``.

## Creating the Platformer inside of the RPG 

Instead of having the RPG lead to somewhere else in the page like we had before, I decided to create another class inside of `GameLevelOverworld.js` that creates the platformer. When the player interacts with the Villager at the end of the RPG level, a canvas is drawn over RPG. So, it's easy to go from the **platformer** to the **RPG** without having any complicated code that transfers from level to level in the Adventure Game.

```js 
this.canvas = document.createElement('canvas');
this.canvas.id = 'platformerMiniCanvas';
this.canvas.width = window.innerWidth;
this.canvas.height = window.innerHeight;
Object.assign(this.canvas.style, {
  position: 'fixed',
  top: '0',
  left: '0',
  zIndex: '10000', 
  backgroundColor: 'rgba(135, 206, 235, 1)' // this is overwritten later, now the background is an image
});
document.body.appendChild(this.canvas);
```
The PlatformerMini creates its own <canvas> element when the `start()` method is called.

- `position: 'fixed'`: This makes the canvas overlay everything on the page, sticking to the top-left corner.
- `zIndex: '10000'`: This ensures the platformer canvas is on top of everything else, including the RPG’s canvas.
- The canvas fills the entire screen (`width` and `height` are set to window.innerWidth and `window.innerHeight`).

```js
pauseRpg();
```

This pauses the RPG game when the platformer starts, so the RPG’s logic is paused underneath.

```js
loop = () => {
        this.update();
        this.draw();
        if (this.isRunning) {
          this.animationFrameId = requestAnimationFrame(this.loop);
        }
      }
```

Because the canvas was added as a full-screen, fixed-position element, the `draw()` method’s drawing happens on top of everything else (like the RPG canvas underneath). This loop continuously draws the platformer visuals on that high-zIndex canvas while the RPG is paused.

```js
if (this.canvas?.parentNode) {
  this.canvas.parentNode.removeChild(this.canvas);
  this.canvas = null;
  this.ctx = null;
}
```

When you die (`stop()` method), or when you exit via NPC or another method, the platformer’s canvas is removed from the DOM.
This ensures the underlying RPG canvas is visible again.



## Platformer Features 

Inside of the actual platform that I have inside of `GameLevelOverworld.js`, I have multiple interesting features that make the game more interesting. First, we'll start with the **collectable item**.

```js
this.collectibleImage = new Image();
        this.collectibleImage.src = `${gameEnv.path}/images/platformer/sprites/sword.png`; // Replace with your image path
```

First, we start with the image that I use for the item. Since the game is Minecraft themed and the enemy is a zombie, the collectable item is a sword. 

```js
if (
  !this.itemCollected &&             // Item not yet collected
  this.playerX + this.playerWidth > this.canvas.width / 2 - 120 && // Player's right edge past item's left edge
  this.playerX < this.canvas.width / 2 - 80 &&  // Player's left edge before item's right edge
  this.playerY + this.playerHeight > this.groundY - 110 && // Player's bottom past item's top edge
  this.playerY < this.groundY - 70 && // Player's top before item's bottom edge
  this.keysPressed['KeyC']           // 'C' key pressed
)
```

If all conditions are true, then .. 

```js 
this.itemCollected = true; // The item is marked as collected
console.log('Item collected!'); // Log a message to the console
```

- Player overlaps the item’s bounding box.
- Player is pressing 'C'.
- The item hasn’t been collected yet.
It sets `this.itemCollected = true` to mark the item as collected and logs “Item collected!”.

Something else with item is that it can be used to kill the enemy! If the player has already picked up the sword and collides with the enemy, the enemy dies. If not, the player dies.

```js
        if ( // collision detection 
          this.playerX + 50 > this.enemyX && // Player's right edge is past enemy's left edge
          this.playerX < this.enemyX + this.enemyWidth && // Player's left edge is before enemy's right edge
          this.playerY + 50 > this.enemyY && // Player's bottom edge is past enemy's top edge
          this.playerY < this.enemyY + this.enemyHeight // Player's top edge is before enemy's bottom edge
        ) {
          if (!this.itemCollected) { // if item is not collected .. 
            this.stop(); // Player dies 
            console.log('Player died!');
          } else { // else, meaning the player does have the item 
            this.enemyX = -100; // Move enemy off-screen to simulate death
            this.enemyDefeated = true; // Mark the enemy as defeated
            console.log('Enemy defeated!');
          }
        }
```

Assuming that the enemy is defeated, the player can now talk to the chicken at the end of the level and return to `GameLevelDesert.js`! But if the enemy isn't defeated, then the player is stuck in that platformer until the enemy is defeated. 

```js
keyDownHandler = (e) => {
        this.keysPressed[e.code] = true;

        // Handle interaction with NPC when 't' is pressed
        if (e.code === 'KeyE') {
          if (
            this.playerX + 50 > this.npcX && // Player's right edge is past NPC's left edge
            this.playerX < this.npcX + this.npcWidth && // Player's left edge is before NPC's right edge
            this.playerY + 50 > this.npcY && // Player's bottom edge is past NPC's top edge
            this.playerY < this.npcY + this.npcHeight // Player's top edge is before NPC's bottom edge
          ) {
            if (!this.enemyDefeated) { // if the enemy is not defeated .. 
              this.showDialogue(
                "BAWK! I'm too scared to move because of that monster over there! ( You have to defeat it first! )",
                "Scared NPC"
              ); // the chicken tells the player that the enemy is still alive, and the player has to go back and kill it 
            } else { // else, aka if the enemy is defeated 
              this.showDialogue(
                "Hooray! You have slain the monster! Let's get out of here.. ( You will now be transported back to the Desert! )",
                "Grateful NPC",
                () => {
                  window.location.reload(); // reloads the page, which simulates bringing the player back to GameLevelDesert 
```